In [120]:
import numpy as np
from scipy.special import expit as sigmoid
from typing import List, Union, Callable
import matplotlib.pyplot as plt
import sys

In [3]:
X = np.identity(8)
y = X

In [265]:
def sigmoid_derivative(z: float) -> float:
    return sigmoid(z)*(1-sigmoid(z))

def quadratic_loss(predictions: np.ndarray, actuals: np.ndarray) -> np.ndarray:
    norms = np.apply_along_axis(np.linalg.norm, 0, predictions-actuals)
    return 0.5*np.apply_along_axis(np.power, 0, norms, 2)
    # return 0.5*np.apply_along_axis(np.power, 0, norms, {'x2': 2})

def cost_function(loss_function: Callable, predictions: np.ndarray, actuals: np.ndarray, weigths: np.ndarray, decay_parameter: float) -> float:
    avg_loss: float = np.mean(loss_function(predictions, actuals))
    regularization: float = 0.5 * decay_parameter * sum(np.power(weights, 2))
    return avg_loss+regularization

class Layer:

    # declaration of instance variables
    weigths: np.ndarray
    has_bias: bool

    def __init__(self, num_nodes: int, num_nodes_n1: int, include_bias: bool = True, epsilon: float = 0.1) -> None:
        self.weights = np.random.normal(loc=0, scale=np.power(epsilon,2), size=(num_nodes_n1, np.add(num_nodes, include_bias)))
        self.has_bias = include_bias
    
    def print_weights(self) -> None:
        print(self.weights)


class Network:

    # declaration of instance variables
    layers: List[Layer]
    weights: List[np.ndarray]

    def __init__(self, num_nodes: List[int], include_biases: List[bool]) -> None:
        # num_nodes is a list of number of nodes for all layers not counting the bias node
        # TO-DO: code won't work if include_biases != [True, True, False], see prop_forward
        assert (include_biases == [True, True, False]), 'error when initializing Network class: include_bias parameter not available'

        self.layers = [Layer(num_nodes[i], num_nodes[i+1], include_biases[i]) for i in range(len(num_nodes)-1)]
        self.layers.append(Layer(num_nodes[-1], 0, include_biases[-1]))
        self.weights = self.get_weights(form='list')

    def get_weights(self, form: str = 'vector') -> Union[List[np.ndarray], np.ndarray]:
        assert (form in ['vector', 'list']), 'Error in get_weights function: form parameter ill-defined'

        if form == 'vector':
            # returns one np.ndarray with all weights of all layers
            weigth_vector = []
            for layer in self.layers:
                weigth_vector.append(layer.weights)
            return np.asarray(weigth_vector)
        elif form == 'list':
            # returns a list, where list[i] stores the weights between layer i-1 and layer i
            list_of_weights: List[np.ndarray] = []
            for layer in self.layers:
                list_of_weights.append(layer.weights)
            return list_of_weights

    def print_weights(self) -> None:
        print('Printing weights of network:')
        for index, layer in enumerate(self.layers):
            print(f'Layer {index+1}')
            layer.print_weights()

    def prop_forward(self, features: np.ndarray) -> List[np.ndarray]:
        # returns a list, where list[i] stores the activations for neurons in layer i+1
        # the activation of a bias node (should the layer have one) is given by the first value in the array and is always =1
        # TO-DO: right now it is hard coded that input layer & hidden layer have a bias node, but output layer has not
        z_2 = np.matmul(self.weights[0], np.append(1, features))
        a_2 = np.apply_along_axis(sigmoid, 0, z_2)
        z_3 = np.matmul(self.weights[1], np.append(1, a_2))
        a_3 = np.apply_along_axis(sigmoid, 0, z_3)
        # print(a_3)
        return [np.append(1, features), np.append(1, a_2), a_3]

    def print_activations(self, features: np.ndarray) -> None:
        print(f'Printing activations for input: {features}')
        for index, array in enumerate(self.prop_forward(features)):
            print(f'Layer {index+1}: {array}')

    def get_deltas(self, X: np.ndarray, y: np.ndarray, activations: List[np.ndarray]=None) -> List[np.ndarray]:
        # TO-DO: adapt code to accept different cost functions
        # Right now: hard coded to use quadratic loss
        if activations == None:
            activations = self.prop_forward(X)
        weights = self.weights
        deltas = []

        # z = activations[-1] * weights[-2]

        deltas_output = -np.multiply((y-activations[-1]), np.apply_along_axis(sigmoid_derivative, 0, np.dot(weights[1], activations[-2])))
        #  print('mat_mul: ', (y-activations[-1]), 'z: ', np.dot(weights[1], activations[-2]))
        deltas.insert(0, deltas_output)

        # print('shape: ', np.transpose(weights[0]).shape, deltas[0].shape, np.matmul(weights[0][:, 1:], deltas[0]))

        mat_mul = np.matmul(weights[1].T, deltas[0].reshape((8,1)))
        z = np.matmul(weights[0], activations[0])
        
        # print('mat_mul: ', mat_mul, 'z: ', z)
        
        deltas_2 = -np.multiply(mat_mul[1:], np.apply_along_axis(sigmoid_derivative, 0, z))
        
        # print(deltas_2)
        
        deltas.insert(0, deltas_2)
        '''
        for i in range(len(activations)-2):
            
            
            
            delta = np.multiply(
                np.matmul(np.transpose(weights[-(i+2)]), deltas[-(i+1)]), np.apply_along_axis(sigmoid_derivative, 0, np.dot(weights[1], activations[-2])))
            
            
            # remove 'bias delta', as activation of bias cannot be changed
        
            deltas.insert(0, delta[1:])
            '''
        print(deltas)
        return deltas

    def partial_derivatives(self, X: np.ndarray, y: np.ndarray, verbose: bool =False) -> List[np.ndarray]:
        activations = self.prop_forward(X)
        deltas = self.get_deltas(X, y, activations)
        partial_derivatives = []
        for index in range(len(deltas)):
            if verbose:
                # for testing/debugging purposes
                print(f'Layer {index+1}: dimension deltas {index+2} {deltas[index].shape}, dimension activations {index+1} {activations[index].shape}')
                print(f'activations: {activations[index]}')
                print(f'deltas: {deltas[index]}')
            partial = np.outer(deltas[index], np.transpose(activations[index]))
            if verbose:
                # deltas should be equal to partial derivatives of the bias node
                print(f'Partial of bias: {partial.shape}')
            partial_derivatives.append(partial)
        return partial_derivatives


    def update_weights(self, big_delta: List[np.ndarray], regularization_parameter: float, learning_rate: float=0.01) -> None:
        for num_layer in range(len(self.weights)):
            self.weights[num_layer] = self.weights[num_layer]-learning_rate*(big_delta[num_layer] + regularization_parameter*self.weights[num_layer])

    def gradient_descent(self, X_train: np.ndarray, y_train: np.ndarray, regularization_parameter: float, learning_rate: float=0.01) -> None:
        # This is only for a single example? 
        # --> create batch gradient descent
        big_delta: List[np.ndarray] = [] 
        for index in range(len(self.weights)):
            big_delta.append(np.zeros(self.weights[index].shape))
        for num_instance in range(X_train.shape[0]):
            partials = self.partial_derivatives(X[num_instance], y[num_instance])
            for num_layer in range(len(self.layers)-1):
                
                print('num_layer: ', num_layer, 'big_delta: ', big_delta[num_layer], 'partials: ', partials[num_layer])
                
                big_delta[num_layer] = big_delta[num_layer]+partials[num_layer]
        for num_layer in range(len(self.layers)):
            big_delta[num_layer] = (1/X_train.shape[0])*big_delta[num_layer]
        self.update_weights(big_delta, regularization_parameter, learning_rate)



    def train_network(self, n_iter: int, X_train: np.ndarray, y_train: np.ndarray, regul_param: float, learning_rate: float=0.01) -> None:
        # TO-DO: print/plot loss after each iteration 
        
        # calculate activation
        # loss based on activation
        costs = [[], []]
        for iter in range(n_iter):
            
            temp_costs = []            
            
            for instance in X_train:
                act_3 = self.prop_forward(instance)[2].reshape((8,1))
                weights_3 = self.weights[-2]
                # print('shape act_3: ', act_3.shape, ' shape weights_3: ', weights_3.shape)
                cos = cost_function(quadratic_loss, act_3, instance, weights_3, 1)
                temp_costs.append(cos)
                # print('predictions: ', act_3)
            costs[0].append(iter)
            costs[1].append(np.mean(temp_costs))
            self.gradient_descent(X_train, y_train, regularization_parameter=regul_param, learning_rate=learning_rate)
            
        
        
        plt.plot(costs[0], costs[1])
        return None


test_network = Network([8,3,8], [True, True, False])

In [266]:
test_network.train_network(10000, X, y, regul_param=0.1, learning_rate=0.5)

mat_mul:  [ 0.50114193 -0.50217193 -0.49777153 -0.49889942 -0.49969396 -0.50236015
 -0.49519601 -0.49928746] z:  [-0.00456773  0.00868777 -0.00891395 -0.00440234 -0.00122416  0.00944065
 -0.01921653 -0.00285017]
mat_mul:  [[-0.00302604]
 [-0.0016963 ]
 [ 0.00040377]
 [ 0.00397432]] z:  [0.03629225 0.00636748 0.00268167]
[array([[ 0.00042394,  0.00042407,  0.00042408],
       [-0.00010091, -0.00010094, -0.00010094],
       [-0.00099325, -0.00099357, -0.00099358]]), array([-0.12528483,  0.12554061,  0.12444041,  0.12472425,  0.12492344,
        0.12558724,  0.12378758,  0.12482161])]
num_layer:  0 big_delta:  [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]] partials:  [[ 0.00042394  0.00042394  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.00042407  0.00042407  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.00042408  0.00042408  0.          0.          0.          0.
   0.  

ValueError: operands could not be broadcast together with shapes (3,9) (9,9) 

In [264]:
# test_network.print_weights()
# test_network.print_activations(X[0])
# print(test_network.get_deltas(X[0], X[0]))
for instance in X:
    print(test_network.partial_derivatives(instance, instance, verbose=True)[0])
# test_network.gradient_descent(X, y, 1)

mat_mul:  [ 0.49799068 -0.49508336 -0.49910857 -0.50165805 -0.49952519 -0.49939136
 -0.49864113 -0.50051377] z:  [ 0.00803734 -0.01966719 -0.00356571  0.00663221 -0.00189925 -0.00243456
 -0.00543549  0.00205507]
mat_mul:  [[-4.86375489e-04]
 [-3.86207036e-03]
 [-3.22409365e-03]
 [ 6.00365745e-06]] z:  [-0.01207075 -0.00675809  0.02142671]
[array([[ 9.65482422e-04,  9.65506566e-04,  9.65406781e-04],
       [ 8.05994053e-04,  8.06014209e-04,  8.05930907e-04],
       [-1.50085969e-06, -1.50089722e-06, -1.50074211e-06]]), array([-0.12449566,  0.12375887,  0.12477675,  0.12541313,  0.12488118,
        0.12484766,  0.12465936,  0.12512831])]
Layer 1: dimension deltas 2 (3, 3), dimension activations 1 (9,)
activations: [1. 1. 0. 0. 0. 0. 0. 0. 0.]
deltas: [[ 9.65482422e-04  9.65506566e-04  9.65406781e-04]
 [ 8.05994053e-04  8.06014209e-04  8.05930907e-04]
 [-1.50085969e-06 -1.50089722e-06 -1.50074211e-06]]
Partial of bias: [ 9.65482422e-04  9.65506566e-04  9.65406781e-04  8.05994053e-04
  8.0

In [54]:
a = np.random.rand(1,8)
b = np.array([1,0,0,0,0,0,0,0])
weights = np.random.rand(5,8)
cost_function(quadratic_loss, a, b, weights, decay_parameter=1)

array([0.75189953, 1.22339985, 1.22547545, 0.79885732, 0.91563999,
       0.96365191, 0.97013856, 1.09595915])

In [267]:
len(np.array([0,1,2]))

3